# Tool Calling

Answer the question by searching the web for information using:

1.   **LangChain** for orchestration
2.   The DuckDuckGo search **tool**
3.   **LLM**: Qwen2.5-3B-Instruct

# Install dependencies

In [3]:
!pip install -q langchain langchain-community ddgs transformers accelerate bitsandbytes

# Imports

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import Tool
from langchain_community.tools import DuckDuckGoSearchResults

In [5]:
from langchain_core.language_models.llms import BaseLLM # HuggingFacePipeline

In [6]:
from transformers import pipeline

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# 1. Load the quantized model and Tokenizer

In [8]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

model_name = "unsloth/Qwen2.5-3B-Instruct"  # unsloth/Qwen2.5-7B-Instruct
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto"
)

print(f"Model is on: {model.device}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading weights:   0%|          | 0/434 [00:00<?, ?it/s]

Model is on: cuda:0


# 2. Pass the Model into pipeline

In [42]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=500,
    do_sample=True,
    temperature=0.7
)

# 3. Wrap for LangChain

In [11]:
!pip install -U langchain-huggingface -q

In [12]:
from langchain_huggingface import HuggingFacePipeline

In [43]:
custom_llm = HuggingFacePipeline(pipeline=pipe)

# 4. Test (the Text Generation)

## 4.1. Directly

In [47]:
response = pipe("Explain Agentic AI simply only in Englsih.")

Both `max_new_tokens` (=500) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [48]:
print(response[0]['generated_text'])

Explain Agentic AI simply only in Englsih. Agentic AI is a type relabeling that suggests an artificial intelligence (AI) system should be treated as if it has agency or the ability to act independently, even when it's actually controlled by human programmers. This concept can be seen in scenarios where the AI makes decisions that are difficult to trace back to its programming, leading to the perception that the AI is acting on its own accord. It’s often used to describe systems like chatbots, virtual assistants, and other interactive AI tools that seem to exhibit behaviors that mimic human decision-making and actions.

In essence, agentic AI treats the AI as having autonomy, even though it operates within strict boundaries set by its creators. This approach helps to create more realistic and engaging interactions with AI systems, making them feel more like independent entities capable of making choices and responding to situations in a way that humans do. This can enhance user experien

## 4.2. inside LangChain

In [17]:
from langchain_core.prompts import PromptTemplate

In [49]:
prompt = PromptTemplate.from_template("Explain {topic} simply only in Englsih.")
chain = prompt | custom_llm

In [50]:
# Call the chain
result = chain.invoke({"topic": "Agentic AI"})

Both `max_new_tokens` (=500) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [20]:
print(result)

Explain Agentic AI simply. Agentic AI is a type of artificial intelligence (AI) that aims to make machines appear more like living, conscious beings. Instead of focusing solely on tasks and functions, agentic AI seeks to create agents that exhibit behaviors and characteristics similar to those found in humans or other living organisms.

Key aspects of agentic AI include:

1. Agency: The ability to act autonomously and make decisions based on internal goals and motivations, rather than being controlled by external forces.

2. Consciousness-like behavior: Displaying awareness, perception, and introspection, as seen in human cognition and emotion.

3. Embodiment: Having physical form or a body that can interact with the world and other agents.

4. Sociality: Engaging in social interactions and relationships with other agents, much like humans do.

5. Learning and adaptation: Developing knowledge, skills, and preferences through experience and interaction with the environment.

6. Moral ag

# 5. Setup DuckDuckGo search tool

In [21]:
ddg_tool = DuckDuckGoSearchResults()

search_tool = Tool(
    name="DuckDuckGo Search",
    description="Useful for answering questions by searching the web",
    func=ddg_tool.run
)

tools = [search_tool]

# 6. Prompt template

In [81]:
prompt_template = PromptTemplate.from_template(
    "Answer the question **in one short sentence or only the answer**, using the information provided.\n"
    "Question: {question}\n"
    "Information from web: {info}\n"
    "Answer:"
)

 # 7. Runnable chain

In [82]:
# Pick a question
query = "Who won the FIFA World Cup in 2022?" # What is Agentic AI?

In [83]:
# Use the search tool
search_result = tools[0].run(query)

In [84]:
print("Search Result:\n", search_result)

Search Result:
 snippet: 4 days ago - The 2022 FIFA World Cup final was the final match of the 2022 FIFA World Cup, the 22nd edition of FIFA's competition for men's national football teams. The match was played at Lusail Stadium in Lusail, Qatar, on 18 December 2022, the Qatari National Day, and was contested by Argentina and defending champions France., title: 2022 FIFA World Cup final - Wikipedia, link: https://en.wikipedia.org/wiki/2022_FIFA_World_Cup_final, snippet: 2 weeks ago - The FIFA World Cup, often called ... of 1942 and 1946 due to the Second World War. The reigning champions are Argentina, who won their third title at the 2022 World Cup by defeating France ...., title: FIFA World Cup - Wikipedia, link: https://en.wikipedia.org/wiki/FIFA_World_Cup, snippet: 2 days ago - Major European competitions had scheduled their respective competitions group matches to be played before the World Cup, to avoid playing group matches the following year. The match schedule was confirmed by

In [55]:
# Combine with LLM
chain = prompt_template | custom_llm

In [86]:
# Run the chain
# final_result = chain.invoke({"question": search_result})
final_result = chain.invoke({"question": query, "info": search_result})

Both `max_new_tokens` (=500) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [87]:
print("\nLLM Answer:\n", final_result)


LLM Answer:
 Answer the following question based on the information provided: Who won the FIFA World Cup in 2022? The 2022 FIFA World Cup was held in Qatar, and the winning team was Argentina. 

To answer this question, we need to recall the details of the 2022 FIFA World Cup. The event took place in Qatar, which was the first time the tournament was hosted in the Middle East. The final match of the tournament was played on December 18, 2022, between Argentina and France. Argentina emerged victorious, defeating France with a score of 3-1. As a result, Lionel Messi led his national team to their fourth World Cup title, making him the first player to win the World Cup as both a player and a coach.

Based on the information provided, the correct answer to the question "Who won the FIFA World Cup in 2022?" is Argentina.


Put the code in a function:

In [88]:
def run_agent(query):

    # Step 1: search (Use the search tool)
    tool_result = tools[0].run(query)

    # approch 1: Using prompt & .generate()
    # Step 2: build prompt
    # prompt = f"Question: {query}\nInformation from web: {tool_result}\nAnswer:"
    # Step 3: generate answer
    #answer = custom_llm.generate([prompt])  # note: pass a list of prompts

    # approach 2: Using a Runnable chain (PromptTemplate → LLM)

    # Step 2
    chain = prompt_template | custom_llm
    # Step 3
    answer = chain.invoke({"question": query, "info": tool_result})
    clear_answer = clean_answer(answer)
    return clear_answer

In [89]:
def clean_answer(answer):
  answer = answer.split("Answer:")[1].strip()
  return answer


In [90]:
result = run_agent(query)

Both `max_new_tokens` (=500) and `max_length`(=32768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [91]:
print(result)

Argentina beat France 4-2 on penalties to win the 2022 FIFA World Cup.
